In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from os.path import join as oj
import math
import pygsheets
import pickle as pkl
import pandas as pd

sys.path.append('../..')
sys.path.append('../../modeling')
import load_data
import exponential_modeling

outcomes = ['tot_cases', 'tot_deaths']
df_hospital = load_data.load_hospital_level(data_dir='../../data_hospital_level', 
                                            merged_hospital_level_info='processed/10_hospital_level_info_merged_with_callnotes.csv',)
df_county = load_data.load_county_level('../../data')
df_county = df_county.sort_values('tot_deaths', ascending=False)
df = df_hospital.merge(df_county, how='left', on='countyFIPS')
df[outcomes] = df[outcomes].fillna(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# rename keys
remap = {
    '#ICU_beds': 'ICU Beds in County',
    'County Name_x': 'County Name',
    'Facility Name_x': 'Hospital Name',
    'tot_deaths': 'Tot Deaths',
    'tot_cases': 'Tot Cases',
}
df = df.rename(columns=remap)
# filter hospitals
df = df[~df['countyFIPS'].isna() & df['ICU Beds'] > 0] # & df['IsAcademicHospital'] & df['ICU Beds'] > 0]
df = df.sort_values(by=['Tot Deaths', 'ICU Beds', 'Total Employees'], ascending=False)
d_contact = df[['Hospital Name', 'County', 'City', 'Website',  
                'Phone Number', 'CMS Certification Number', 'countyFIPS',
                'Donation Phone Number', 'Donation Email', 
                'Type (donation)', 'Notes']].fillna('').replace('NaN', '')

# export to google sheets

- requires a creds.json file in the root directory
- need to make a sheet and share it with `chandan@r4l-predictions.iam.gserviceaccount.com`
- contact info spreadsheet: https://docs.google.com/spreadsheets/d/13VKYtpZvHaeRAZ63tpKCLGoCBLpxTAP2pbMYWnZjKAs/edit#gid=0

In [17]:
gc = pygsheets.authorize(service_file='../../creds.json')

#open the google spreadsheet (where 'PY to Gsheet Test' is the name of my sheet)
sheet_name = 'Contact info'
sh = gc.open(sheet_name) # name of the hospital

# ventilator
wks = sh[0] #select a sheet
wks.update_value('A1', "Note: this sheet is read-only (automatically generated by the data and model)")
wks.set_dataframe(d_contact, (3, 1)) #update the first sheet with df, starting at cell B2. 